# Clustering Crypto

In [72]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [73]:
# Load the crypto_data.csv dataset.
file_path = 'crypto_data.csv'
crypto_df = pd.read_csv(file_path,index_col=[0])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [74]:
# Keep all the cryptocurrencies that are being traded.

# define get_unique_values function to get unique value from a column
def get_unique_values(member):
    value_list = []
    for value in member:
        if value not in value_list:
            value_list.append(value)
        else:
            continue
    return value_list

In [75]:
# call get_unique_values function to keep all cryptocurrencies being traded
crypto_list = get_unique_values(member=crypto_df['CoinName'])
print(crypto_list[0:5])

['42 Coin', '365Coin', '404Coin', 'SixEleven', '808']


In [76]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop('IsTrading', axis=1)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [77]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
crypto_df.isna().sum()


CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [78]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [79]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df = crypto_df[['CoinName']].copy(deep=True)
crypto_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [80]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithcrypto_df = crypto_df.drop('CoinName', axis=1)
crypto_df = crypto_df.drop(columns='CoinName', axis=1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [81]:
# Use get_dummies() to create variables for text features.
X= pd.get_dummies(crypto_df, columns=['Algorithm','ProofType'], drop_first=True)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
print(X_scale)

[[-0.11450078 -0.15072489 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.09041991 -0.14255828 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [ 0.55369883  4.67185916 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 ...
 [-0.11433426 -0.15040252 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.11449785 -0.15070954 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]
 [-0.11401032 -0.14918981 -0.04166667 ... -0.04166667 -0.04166667
  -0.04166667]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [83]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scale)

In [84]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=X_pca, columns=['principal component 1','principal component 2', 'principal component 3']
)
pcs_df.head()

,principal component 1,principal component 2,principal component 3
0,-0.333886,1.210318,-0.509080
1,-0.316550,1.210607,-0.509477
2,2.422652,1.740883,-0.557343
3,-0.133989,-1.381925,0.174396
4,-0.131683,-2.156762,0.346989


In [85]:
print(f'The Explained Variance Ratio for Each Column is: {pca.explained_variance_ratio_}')
print(f'The Total Data Retention is {pca.explained_variance_ratio_.sum()}')

The Explained Variance Ratio for Each Column is: [0.02762719 0.02104543 0.02028884]
The Total Data Retention is 0.0689614715720055


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [86]:
# Create an elbow curve to find the best value for K.
def get_elbow_plot(data):
    inertia = []
    k = range(1,11)

    for i in k:
        km = KMeans(n_clusters=i, random_state=32)
        km.fit(data)
        inertia.append(km.inertia_)

    elbow_data_dict = {'k':k, 'inertia':inertia}
    elbow_df = pd.DataFrame(elbow_data_dict)
    
    plot = elbow_df.hvplot(x='k', y='inertia')

    return plot

In [87]:
elbow_plot = get_elbow_plot(pcs_df)
elbow_plot

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [88]:
# Define get_clusters function
def get_clusters(k, data):
    data = data.copy()

    # Initialize the K-Means model.
    # YOUR CODE HERE
    model = KMeans(n_clusters=k, random_state=56)
    # Fit the model
    # YOUR CODE HERE
    model.fit(data)
    # Predict clusters
    # YOUR CODE HERE
    predictions = model.predict(data)
    data['class'] = model.labels_

    return data

In [89]:
four_clusters = get_clusters(k=4, data=pcs_df)
four_clusters.head()

,principal component 1,principal component 2,principal component 3,class
0,-0.333886,1.210318,-0.509080,0
1,-0.316550,1.210607,-0.509477,0
2,2.422652,1.740883,-0.557343,0
3,-0.133989,-1.381925,0.174396,3
4,-0.131683,-2.156762,0.346989,3


In [90]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.

clustered_df = pd.concat([crypto_df.reset_index(drop=True), four_clusters.reset_index(drop=True)], axis=1, ignore_index=True)

#correct column and index names
clustered_df.columns = pd.concat([crypto_df, four_clusters], axis=1).columns 
clustered_df.index = crypto_df.index

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_name_df['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# already added as part of four_clusters df 

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(577, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333886,1.210318,-0.509080,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.316550,1.210607,-0.509477,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.422652,1.740883,-0.557343,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.133989,-1.381925,0.174396,3,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.131683,-2.156762,0.346989,3,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.167873,-1.018665,-0.029934,3,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.382144,1.256288,-0.440377,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.137456,-2.254112,0.340363,3,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.130066,-2.156852,0.346966,3,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.164516,-2.076062,0.306785,3,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [91]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='principal component 1',
    y='principal component 2',
    z='principal component 3',
    color='class',
    symbol='class',
    hover_name='CoinName',
    hover_data=['Algorithm'],
    width=800
)

fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [92]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [93]:
# Print the total number of tradable cryptocurrencies
print(len(clustered_df))

577


In [94]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
min_max_scaler = MinMaxScaler()
clustered_df_scale = min_max_scaler.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])

plot_df = pd.DataFrame(clustered_df_scale, columns=['TotalCoinSupply','TotalCoinsMined'])

In [95]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df.index = clustered_df.index

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['class'] = clustered_df['class']
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [96]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='class', hover='class')


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)